# Model creation stage III (4x8)

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from tools import common_functions as cf
from tools import regression as reg
from tools import plotting_functions as pf

from tools import sensor_gui as sg

import os
import dill

ARRAY_SIZE = 32

In [2]:
save_path = os.path.join(os.getcwd(), '..', 'models', "4x8")
models = []
    
for i in range(ARRAY_SIZE):

    with open(os.path.join(save_path, f"taxel{i}"), 'rb') as f:
        model = dill.load(f)
        models.append(model)

In [3]:
save_path = os.path.join(os.getcwd(), '..', 'models', "4x8")
models = []
    
for i in range(ARRAY_SIZE):

    with open(os.path.join(save_path, f"taxel{i}"), 'rb') as f:
        model = dill.load(f)
        models.append(model)

In [4]:
import numpy as np

def numpy_to_c_array(np_array, array_name):
    """
    Convert a NumPy array to a C array representation.
    
    Parameters:
    - np_array: NumPy array to be converted
    - array_name: Name of the C array
    
    Returns:
    - A string representing the C code for the array
    """
    if not isinstance(np_array, np.ndarray):
        raise TypeError("Input should be a NumPy array")
    
    array_type = np_array.dtype
    if array_type == np.int32 or array_type == np.int64:
        c_type = 'int'
    elif array_type == np.float32:
        c_type = 'float'
    elif array_type == np.float64:
        c_type = 'double'
    else:
        raise ValueError(f"Unsupported dtype: {array_type}")
    
    # Flatten the array and format it
    flattened_array = np_array.flatten()
    array_elements = ", ".join(map(str, flattened_array))
    
    # Determine the shape of the array for multi-dimensional arrays
    shape = np_array.shape
    if len(shape) == 1:
        c_array = f"{c_type} {array_name}[] = {{{array_elements}}};"
    else:
        dimensions = "][".join(map(str, shape))
        c_array = f"{c_type} {array_name}[{dimensions}] = {{{array_elements}}};"
    
    return c_array

In [5]:
def transform_to_c_strings(expressions):
    result = []
    
    for expression in expressions:
        terms = expression.split()  # Split the expression into terms
        transformed_terms = []
        
        for term in terms:
            if '^' in term:
                base, exponent = term.split('^')
                transformed_terms.extend([base] * int(exponent))
            else:
                transformed_terms.append(term)
        
        result.append(' * '.join(transformed_terms))
    
    return result

# Example usage:
expressions = ['1', 'x0', 'x1', 'x2', 'x0^2', 'x0 x1', 'x0 x2', 'x1^2', 'x1 x2', 'x2^2', 
               'x0^3', 'x0^2 x1', 'x0^2 x2', 'x0 x1^2', 'x0 x1 x2', 'x0 x2^2', 'x1^3', 
               'x1^2 x2', 'x1 x2^2', 'x2^3']

transformed = transform_to_c_strings(expressions)
for expr in transformed:
    print(expr)


1
x0
x1
x2
x0 * x0
x0 * x1
x0 * x2
x1 * x1
x1 * x2
x2 * x2
x0 * x0 * x0
x0 * x0 * x1
x0 * x0 * x2
x0 * x1 * x1
x0 * x1 * x2
x0 * x2 * x2
x1 * x1 * x1
x1 * x1 * x2
x1 * x2 * x2
x2 * x2 * x2


In [6]:
output = ""

# Taxel coefs
for i in range(ARRAY_SIZE):
    output += numpy_to_c_array(models[i].steps[1][1].coef_, f"taxel{i}_coef") + "\n"

# Now function to generate polynomial featuers
feature_names = transform_to_c_strings(models[0].steps[0][1].get_feature_names_out())
output += f"\nvoid generate_polynomial_features(double X0, double Y0, double Z0, double* output) {'{'}\n"
for i, feature in enumerate(feature_names):
    output += f"    output[{i}] = {feature};\n"
output += "}\n"

# Now function to apply the model
output += f"\nvoid apply_model(double* coefs, double* features, double* output) {'{'}\n"
output += "    for (int i = 0; i < 3; i++) {\n"
output += "        output[i] = 0;\n"
output += "        for (int j = 0; j < 32; j++) {\n"
output += "            output[i] += coefs[j] * features[j];\n"
output += "        }\n"
output += "    }\n"
output += "}\n"

print(output)

double taxel0_coef[3][20] = {0.011760910438224312, 0.0009617116226385234, 0.0005484523158539542, 90539.80678658892, 2.066444080685677e-07, -1.360848855545994e-07, -29.253996445045175, -6.108113389037568e-08, 27.34968064870995, -1.5259401834475477, -5.5572564466813015e-11, 2.7717945463097924e-11, 0.01550371905714812, 2.0248816543228236e-11, -0.005598259685232237, 0.1531947706642824, -3.811602465330437e-11, -0.002081831464425939, -27.95868570091862, 2.4261685739118708e-05, 0.028032602654472084, -0.00015738179416161893, 0.0017045335746742193, -17422.71153958339, 1.222488864694403e-08, -1.4632921836594593e-07, -1.9841298327988799, -4.708362392929696e-08, 35.96509742467156, 0.29363917696568453, 1.3807007649329727e-11, 3.1784678847986316e-12, -0.002912718100738204, 2.9725903838841316e-12, -0.008681684443902708, -0.02903393376908675, -1.0890955071208513e-11, -0.006095318688258447, 5.379316191110174, -4.668717857933005e-06, 0.17412302822048262, -0.00027852932851600717, -0.0006603867226588403, 

In [24]:
def generate_polynomials(X0, Y0, Z0, expressions):

    features = []
    for expr in expressions:
        features.append(eval(expr))
  
    return features

def apply_model(coefs, features):
    x, y = coefs.shape
    
    output = np.zeros(x)
    for i in range(x):
        for j in range(y):
            output[i] += coefs[i, j] * features[j]
    return output 

In [25]:
polys = generate_polynomials(1.4, 1.91, 0.00035, transform_to_c_strings(models[0].steps[0][1].get_feature_names_out()))
print(models[0].steps[1][1].coef_.shape)
output = apply_model(models[0].steps[1][1].coef_, polys)

print(models[0].steps[1][1].coef_)

print(output)

(3, 20)
[[ 1.17609104e-02  9.61711623e-04  5.48452316e-04  9.05398068e+04
   2.06644408e-07 -1.36084886e-07 -2.92539964e+01 -6.10811339e-08
   2.73496806e+01 -1.52594018e+00 -5.55725645e-11  2.77179455e-11
   1.55037191e-02  2.02488165e-11 -5.59825969e-03  1.53194771e-01
  -3.81160247e-11 -2.08183146e-03 -2.79586857e+01  2.42616857e-05]
 [ 2.80326027e-02 -1.57381794e-04  1.70453357e-03 -1.74227115e+04
   1.22248886e-08 -1.46329218e-07 -1.98412983e+00 -4.70836239e-08
   3.59650974e+01  2.93639177e-01  1.38070076e-11  3.17846788e-12
  -2.91271810e-03  2.97259038e-12 -8.68168444e-03 -2.90339338e-02
  -1.08909551e-11 -6.09531869e-03  5.37931619e+00 -4.66871786e-06]
 [ 1.74123028e-01 -2.78529329e-04 -6.60386723e-04  3.05102536e+05
  -3.91832508e-07  6.79801515e-08 -1.93860221e+00 -2.62942443e-07
  -2.19876508e+01 -5.14214006e+00 -5.47329481e-11  2.47891929e-12
  -8.52007740e-03 -6.64680692e-11  8.06860292e-04  5.11686162e-01
  -9.53406793e-13 -4.85899049e-04 -9.42132454e+01  8.17574713e-05]